In [ ]:
import pandas as pd 
import numpy as np 
from datasets import load_dataset
import importlib
import re
from datetime import datetime, timezone
from collections import Counter


#project dependences
import utils.text_functions as tf
importlib.reload(tf)

<module 'utils.text_functions' from '/home/onyxia/work/Modality-aware-transformer/utils/text_functions.py'>

Data from the [FNSPID dataset](https://huggingface.co/datasets/Zihan1004/FNSPID).


In [ ]:
# visualisation test (pas besoin de run)

ds = load_dataset("Zihan1004/FNSPID", streaming=True) # true pour pas tout télécharger en local
#print(ds) 

split_name = list(ds.keys())[0]
it = iter(ds[split_name])

samples = [next(it) for _ in range(3)]
#print("Keys:", samples[0].keys())
#print("Example:", samples[0])

Repo card metadata block was not found. Setting CardData to empty.


In [16]:
ds = load_dataset("Zihan1004/FNSPID", streaming=True)
split = ds["train"]

split = split.map(tf.make_projector())
split_clean = split.map(tf.build_text_record)
split_clean = split_clean.filter(tf.is_valid_record)

split_clean = split_clean.map(tf.add_effective_date)
split_clean = split_clean.filter(lambda ex: ex["effective_date"] is not None)

it_dedup = tf.dedup_stream(iter(split_clean), max_seen=500_000)

for _ in range(3):
    ex = next(it_dedup)
    print(ex["dt_utc"], "->", ex["effective_date"], ex["stock_symbol"])

Repo card metadata block was not found. Setting CardData to empty.


2020-06-05 06:30:54+00:00 -> 2020-06-08 A
2020-06-03 06:45:20+00:00 -> 2020-06-04 A
2020-05-26 00:30:07+00:00 -> 2020-05-27 A


In [ ]:
# la plupart des heures est à 00h00, le reste à des heures peut-être pas fiables 
# donc meilleur de partir du fait que une news est associé à un jour et à un impact sur les jours suivants

hours = Counter()

for i, ex in enumerate(split_clean):
    dt = ex.get("dt_utc")
    if dt is not None:
        hours[dt.hour] += 1
    if i > 500_000:   # échantillon suffisant
        break

print(hours)